In [1]:
import json
from tensorflow.keras import optimizers
import numpy as np

In [ ]:
data = []
endata = []
chdata = []

def chine(string):
    for ch in string :
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

for line in open('train.json', 'r', encoding='utf-8'):
    data.append(json.loads(line))  


print('----------')

for i in range(len(data)):
    if chine(data[i]['chinese']) == True :
        if len(data[i]['chinese']) < 10:
            endata.append(data[i]['english'])
            chdata.append('@'+data[i]['chinese']+'。')
        if len(chdata) == 100 :
            break

en_vocab = set(''.join(endata))
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}

# 分別生成中英文字典
ch_vocab = set(''.join(chdata))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}

print('英文數據:\n', endata)
print('\n中文數據:\n', chdata)
# print('\n英文字典:\n', en2id)
# print('\n中文字典共計\n:', ch2id)


# en_vocab = set(''.join(endata))
# id2en = list(en_vocab)
# en2id = {c:i for i,c in enumerate(id2en)}